# 20 – Wildfire Risk Modeling

In this notebook we:

1. Prepare features and labels from the wildfire dataset.
2. Train baseline and tree-based models to predict `fire_occurred`.
3. Evaluate the models using accuracy, precision, recall, F1, and ROC-AUC.

This is a tabular risk model – the same ideas can be extended to image-based
models (e.g., CNNs on satellite imagery) in a separate notebook.

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)

base_dir = "/mnt/data/cardiff_ai_talk_runbook"
data_path = os.path.join(base_dir, "data", "raw", "wildfire_synthetic.csv")
df = pd.read_csv(data_path)

feature_cols = [
    "temp_c",
    "humidity",
    "wind_speed",
    "rain_mm_last_7d",
    "vegetation_index",
    "population_density",
    "month",
    "year",
]
X = df[feature_cols]
y = df["fire_occurred"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

X_train.shape, X_test.shape

In [ ]:
# Baseline: Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred_lr = log_reg.predict(X_test)
y_prob_lr = log_reg.predict_proba(X_test)[:, 1]

def print_classification_metrics(y_true, y_pred, y_prob, label):
    print(f"\n=== {label} ===")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall   :", recall_score(y_true, y_pred))
    print("F1       :", f1_score(y_true, y_pred))
    print("ROC-AUC  :", roc_auc_score(y_true, y_prob))

print_classification_metrics(y_test, y_pred_lr, y_prob_lr, "Logistic Regression")

In [ ]:
# Tree-based model: Random Forest
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    min_samples_split=4,
    random_state=42,
    n_jobs=-1,
)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]

print_classification_metrics(y_test, y_pred_rf, y_prob_rf, "Random Forest")

In [ ]:
# Feature importances from Random Forest
import matplotlib.pyplot as plt
import numpy as np

importances = rf.feature_importances_
idx = np.argsort(importances)[::-1]

plt.bar(range(len(feature_cols)), importances[idx])
plt.xticks(range(len(feature_cols)), np.array(feature_cols)[idx], rotation=45, ha="right")
plt.ylabel("Importance")
plt.title("Random Forest Feature Importances")
plt.tight_layout()
plt.show()